<a href="https://colab.research.google.com/github/medalidia/apprentissage_git/blob/main/projet_polarite_avis_clients.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Projet NLP – Classification de la polarité des avis clients
**Auteur : Mohamed Ali Dia**  
**Date : Juillet 2025**  

Ce projet a pour objectif de développer un système automatique permettant de classifier un avis client en ligne comme **positif** ou **négatif**, en utilisant un jeu de données d’avis en français.  
Nous allons passer par les étapes suivantes :

- Prétraitement des textes (nettoyage, lemmatisation)
- Analyse exploratoire
- Vectorisation des textes (TF-IDF)
- Entraînement de plusieurs modèles de classification
- Évaluation comparative
- Sauvegarde des modèles


## 1. Importation des bibliothèques

In [22]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import nltk
import joblib
import spacy
from wordcloud import WordCloud
from collections import Counter
import itertools

from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    ConfusionMatrixDisplay,
    f1_score
)


## 2. Préparation des outils linguistiques

In [23]:

nltk.download('stopwords')
stop_words = set(stopwords.words('french'))

# Chargement du modèle SpaCy français avec gestion d’erreur
import subprocess
try:
    nlp = spacy.load("fr_core_news_sm")
except:
    subprocess.run(["python", "-m", "spacy", "download", "fr_core_news_sm"])
    nlp = spacy.load("fr_core_news_sm")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 3. Chargement du jeu de données

In [24]:

from datasets import load_dataset

# Télécharger et convertir en DataFrame
dataset = load_dataset("tblard/allocine", split="train")
df = pd.DataFrame(dataset).sample(10000, random_state=42)[['review', 'label']]
df.head()


ValueError: Invalid pattern: '**' can only be an entire path component

## 4. Nettoyage et lemmatisation des textes

In [ ]:

def clean_and_lemmatize(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc
              if token.lemma_.lower() not in stop_words and token.is_alpha and len(token) > 2]
    return " ".join(tokens)

df['clean_review'] = df['review'].apply(clean_and_lemmatize)
df[['review', 'clean_review']].head()


## 5. Exploration des données (répartition des classes)

In [ ]:

print(df['label'].value_counts())

sns.countplot(x='label', data=df)
plt.title("Distribution des avis")
plt.xlabel("Label (0 = négatif, 1 = positif)")
plt.ylabel("Nombre d'exemples")
plt.show()


## 6. Visualisation des mots les plus fréquents

In [ ]:

all_words = list(itertools.chain(*df['clean_review'].str.split()))
word_freq = Counter(all_words).most_common(20)
words, counts = zip(*word_freq)

sns.barplot(x=counts, y=words)
plt.title("20 mots les plus fréquents")
plt.xlabel("Fréquence")
plt.show()

wordcloud = WordCloud(width=800, height=400, background_color="white").generate(" ".join(all_words))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title("Nuage de mots")
plt.show()


## 7. Vectorisation des textes (TF-IDF)

In [ ]:

vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X = vectorizer.fit_transform(df['clean_review'])
y = df['label']


## 8. Séparation des données d'entraînement et de test

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## 9. Entraînement des modèles de classification

In [ ]:

lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)

svm_model = LinearSVC()
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)


## 10. Évaluation des modèles

In [ ]:

def eval_model(name, y_true, y_pred):
    print(f"=== {name} ===")
    print(confusion_matrix(y_true, y_pred))
    print(classification_report(y_true, y_pred))
    print("Accuracy:", accuracy_score(y_true, y_pred))
    ConfusionMatrixDisplay.from_predictions(y_true, y_pred, display_labels=["Négatif", "Positif"])
    plt.title(f"{name} - Matrice de confusion")
    plt.show()

eval_model("Régression Logistique", y_test, y_pred_lr)
eval_model("SVM Linéaire", y_test, y_pred_svm)
eval_model("Random Forest", y_test, y_pred_rf)


### 🔍 F1-score pondéré (complément d'évaluation)

In [ ]:

print("F1-score pondéré - Logistic Regression :", f1_score(y_test, y_pred_lr, average="weighted"))
print("F1-score pondéré - SVM :", f1_score(y_test, y_pred_svm, average="weighted"))
print("F1-score pondéré - Random Forest :", f1_score(y_test, y_pred_rf, average="weighted"))


## 11. Comparaison des performances des modèles

In [ ]:

models = ['Logistic Regression', 'SVM', 'Random Forest']
accuracies = [
    accuracy_score(y_test, y_pred_lr),
    accuracy_score(y_test, y_pred_svm),
    accuracy_score(y_test, y_pred_rf)
]

sns.barplot(x=models, y=accuracies)
plt.title("Comparaison des précisions")
plt.ylabel("Accuracy")
plt.ylim(0, 1)
plt.show()


## 12. Échantillon de prédictions

In [ ]:

indices = y_test.iloc[:5].index
sample_preds = pd.DataFrame({
    "Review": df.loc[indices, 'review'].values,
    "Label réel": y_test.loc[indices].values,
    "Prédiction LR": y_pred_lr[:5],
    "Prédiction SVM": y_pred_svm[:5],
    "Prédiction RF": y_pred_rf[:5]
})
sample_preds


## 13. Sauvegarde des modèles entraînés

In [ ]:

joblib.dump(vectorizer, "tfidf_vectorizer.pkl")
joblib.dump(lr_model, "logistic_model.pkl")
joblib.dump(svm_model, "svm_model.pkl")
joblib.dump(rf_model, "random_forest_model.pkl")

pipeline_lr = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000, ngram_range=(1, 2))),
    ('clf', LogisticRegression(max_iter=1000))
])
pipeline_lr.fit(df['clean_review'], df['label'])
joblib.dump(pipeline_lr, "pipeline_logistic.pkl")

print("Tous les modèles ont été sauvegardés avec succès.")



## 14. Conclusion et perspectives

### ✅ Résumé
Le système de classification automatique des avis clients permet d'obtenir de très bonnes performances, notamment avec la **régression logistique** et le **SVM** qui affichent des scores de précision élevés.  
Le prétraitement des textes (lemmatisation, nettoyage) et l'utilisation de la vectorisation **TF-IDF** se sont révélés efficaces sur ce jeu de données.

### 🚀 Pistes d'amélioration possibles
Voici quelques améliorations pour renforcer ce système :
- **Utiliser des embeddings sémantiques** comme FastText ou BERT pour capturer davantage le sens des mots.
- **Optimiser les hyperparamètres** via `GridSearchCV` ou `RandomizedSearchCV`.
- **Gérer le déséquilibre des classes** avec une pondération (`class_weight`) ou de la suréchantillonnage (SMOTE).
- **Explicabilité des modèles** avec SHAP ou LIME pour mieux comprendre les prédictions.
- **Créer une API Flask** ou une interface simple pour tester des avis en temps réel.
